In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
model_path = 'data/imdb/models'

In [4]:
from keras.datasets import imdb
idx = imdb.get_word_index()

In [5]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [6]:
idx2word = {v: k for k, v in idx.iteritems()}

In [7]:
print(type(idx2word))

<type 'dict'>


In [8]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [9]:
len(x_train)

25000

In [10]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [11]:
idx2word[23022]

'bromwell'

In [13]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [14]:
# The first 10
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [15]:
vocab_size = 5000

trn = [np.array([i if i < vocab_size - 1 else vocab_size - 1 for i in s]) for s in x_train]
test = [np.array([i if i < vocab_size - 1 else vocab_size - 1 for i in s]) for s in x_test]

In [16]:
lens = np.array(map(len, trn))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

In [19]:
# Each sentence is padded to make it a specific length to make it consistent
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

In [20]:
# The results of the training matrices are of length 500.
# Reviews less than 500 words are padded with zeros
# Reviews with more than 500 words are truncated to 500 words.
trn.shape

(25000, 500)

In [21]:
# Creating Simple models
model = Sequential([
        # 32 embeddings.
        Embedding(vocab_size, 32, input_length=seq_len),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.7),
        Dense(1, activation='sigmoid')
    ])

In [22]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 16000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           1600100     flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [23]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 4s - loss: 0.4584 - acc: 0.7584 - val_loss: 0.3264 - val_acc: 0.8570
Epoch 2/2
25000/25000 [==============================] - 3s - loss: 0.1955 - acc: 0.9279 - val_loss: 0.3178 - val_acc: 0.8660


In [26]:
# CNN works better as it takes advantage of ordered data.
conv1 = Sequential([
        Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
        Dropout(0.2),
        Convolution1D(64, 5, border_mode='same', activation='relu'),
        Dropout(0.2),
        MaxPooling1D(),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.7),
        Dense(1, activation='sigmoid')
    ])

In [28]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [29]:
# 89.2 is better than the stanford state of the art result.
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 12s - loss: 0.5074 - acc: 0.7154 - val_loss: 0.2965 - val_acc: 0.8808
Epoch 2/4
25000/25000 [==============================] - 12s - loss: 0.2960 - acc: 0.8799 - val_loss: 0.2763 - val_acc: 0.8839
Epoch 3/4
25000/25000 [==============================] - 11s - loss: 0.2626 - acc: 0.8971 - val_loss: 0.2620 - val_acc: 0.8920
Epoch 4/4
25000/25000 [==============================] - 11s - loss: 0.2388 - acc: 0.9059 - val_loss: 0.2957 - val_acc: 0.8754


In [30]:
conv1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 500, 32)       160000      embedding_input_3[0][0]          
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 500, 32)       0           embedding_3[0][0]                
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 500, 64)       10304       dropout_5[0][0]                  
____________________________________________________________________________________________________
dropout_6 (Dropout)              (None, 500, 64)       0           convolution1d_2[0][0]            
___________________________________________________________________________________________

In [31]:
conv1.save_weights(model_path + 'conv1.h5')

In [32]:
conv1.load_weights(model_path + 'conv1.h5')

In [33]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [34]:
vecs, words, wordidx = load_vectors('data/glove/results/6B.50d')

In [35]:
# Creating an embedding matrix.

def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))
    
    for i in range(1, len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            emb[i] = normal(scale=0.6, size=(n_fact,))
    
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [36]:
emb = create_emb()

In [37]:
# 50 dimensional vectors and the vocabulary size is 5000
emb.shape

(5000, 50)

In [39]:
model = Sequential([
        Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, weights=[emb], trainable=False),
        Dropout(0.25),
        Convolution1D(64, 5, border_mode='same', activation='relu'),
        Dropout(0.25),
        MaxPooling1D(),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.7),
        Dense(1, activation='sigmoid')
    ])

In [40]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [43]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=3, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 12s - loss: 0.4400 - acc: 0.7962 - val_loss: 0.4101 - val_acc: 0.8192
Epoch 2/3
25000/25000 [==============================] - 12s - loss: 0.4360 - acc: 0.7988 - val_loss: 0.4210 - val_acc: 0.8119
Epoch 3/3
25000/25000 [==============================] - 12s - loss: 0.4223 - acc: 0.8051 - val_loss: 0.3999 - val_acc: 0.8298


In [44]:
model.layers[0].trainable=True

In [46]:
model.optimizer.lr = 1e-4

In [47]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 12s - loss: 0.4199 - acc: 0.8102 - val_loss: 0.4064 - val_acc: 0.8188
